# Equal Weighted Portfolio


## Imports

In [ ]:
import pandas as pd
import numpy as np
import yfinance as yf
from matplotlib import pyplot as plt
from matplotlib.ticker import PercentFormatter
# plt.style.use('seaborn-v0_8')

from datasets.datasets import get_sp500_tickers, load_data
from trading_algos.utils import calc_returns
from optimization.optimization import select_n

%load_ext autoreload
%autoreload 2

## Data

In [ ]:

# Selecting 5 stocks at random
tickers = select_n(5,
                   get_sp500_tickers(get_latest=True),
                   seed=1)
tickers

In [ ]:
# Load selecting stock data
df_stocks = load_data(tickers=tickers,
                      start_date='2010-01-01',
                      end_date='2025-01-01')
df_stocks.head()

## Viz

In [ ]:
fig, ax = plt.subplots(figsize=(12,6))

ax.set_title('Normalized Returns')

ax.plot(df_stocks.Close.div(df_stocks.Close.iloc[0]).mul(100).sub(100).rolling(1).mean())
ax.legend(labels=df_stocks.Close.columns)
ax.grid(alpha=0.3)
ax.spines[['top', 'right']].set_visible(False)
ax.yaxis.set_major_formatter(PercentFormatter())
plt.show();

In [ ]:
fig, ax = plt.subplots(figsize=(12,6))

ax.set_title('Stock Price')

ax.plot(df_stocks.Close.rolling(1).mean())
ax.legend(labels=df_stocks.Close.columns)
ax.grid(alpha=0.3)
# ax.spines()
plt.show();

In [ ]:
df_stocks.div(df_stocks.iloc[0]).rolling('30D').mean().plot(figsize=(12,6))
plt.show();

In [ ]:
df_stocks.plot(figsize=(12,8))

## Creating the EWP

In [ ]:
# Calculating the daily returns of each stock

df_returns = df_stocks.pct_change().dropna()
df_returns.head()

In [ ]:
from trading_algos.utils import calc_returns

calc_returns(df_stocks)

In [ ]:
# Each asset is given the same monetary investment equal to 1/N of the total investment amount

num_assets = len(df_stocks.columns)
weights = [1/num_assets for i in range(num_assets)]

In [ ]:
# We can calculate the equal weighted portfolio returns by refactoring the returns by the equal weight and summing them together i.e. the dot product

df_returns['EWP'] = df_returns.dot(weights)
df_returns.head()

In [ ]:
from optimization.optimization import calculate_weights
df_returns.dot(calculate_weights(df_stocks))

In [ ]:
(1+df_returns).resample('ME').mean().cumprod().plot(figsize=(12,8))

In [ ]:
summary = df_returns.agg(['mean', 'std']).T
summary.columns = ['Return', 'Risk']
summary

In [ ]:
ann_summary = summary.copy()
ann_summary['Return'] = ann_summary['Return'] * 252
ann_summary['Risk'] = ann_summary['Risk'] * np.sqrt(252)
ann_summary['Sharpe'] = (ann_summary['Return'] - 0) / ann_summary['Risk']
ann_summary

In [ ]:
ann_summary.plot(kind = "scatter", x = "Risk", y = "Return", figsize = (13,9), s = 50, fontsize = 15)
for i in ann_summary.index:
    plt.annotate(i, xy=(ann_summary.loc[i, "Risk"]+0.002, ann_summary.loc[i, "Return"]+0.002), size = 15)
plt.xlabel("ann. Risk", fontsize = 15)
plt.ylabel("ann. Return", fontsize = 15)
plt.title("Risk/Return", fontsize = 20)
plt.show()